# 4. Onboarding Faber, Acme, Thrift and Government by Steward

**Faber, Acme, Thrift and Government should now establish a Connection with the Steward.**

Each connection is actually a pair of Pairwise-Unique Identifiers (DIDs). The one DID is owned by one party to the connection and the second by another.

Both parties know both DIDs and understand what connection this pair describes.

The relationship between them is not shareable with others; it is unique to those two parties in that each pairwise relationship uses different DIDs.

We call the process of establish a connection **Onboarding**.

In this tutorial we will describe the simple version of onboarding process.
In our case, one party will always be the Trust Anchor. Real enterprise scenarios can use a more complex version.

##### Connecting the Establishment
Let's look the process of connection establishment between **Steward** and **Faber College**.

1. **Faber** and **Steward** contact in a some way to initiate onboarding process.
   It can be filling the form on web site or a phone call.

2. **Steward** creates a new DID record in the wallet by calling ``did.create_and_store_my_did`` that he will use for secure interactions only with **Faber**.
    ```python
    # Steward Agent
    (steward_faber_did, steward_faber_key) = await did.create_and_store_my_did(steward_wallet, "{}")
    ```

3. **Steward** sends the corresponding `NYM` transaction to the Ledger by consistently calling ``ledger.build_nym_request`` to build the NYM request and ``ledger.sign_and_submit_request`` to send the created request.
    ```python
    # Steward Agent
    nym_request = await ledger.build_nym_request(steward_did, steward_faber_did, steward_faber_key, None, role)
    await ledger.sign_and_submit_request(pool_handle, steward_wallet, steward_did, nym_request)
    ```

4. **Steward** creates the connection request which contains the created `DID` and `Nonce`.
   This nonce is just a big random number generated to track the unique connection request. A nonce is a random arbitrary number that can only be used one time. When a connection request is accepted, the invitee digitally signs the nonce so that the inviter can match the response with a prior request.
    ```python
    # Steward Agent
    connection_request = {
        'did': steward_faber_did,
        'nonce': 123456789
    }
    ```

5. **Steward** sends the connection request to **Faber**.

6. **Faber** accepts the connection request from **Steward**.

7. **Faber** creates a wallet if it does not exist yet.
    ```python
    # Faber Agent
    await wallet.create_wallet(pool_name, 'faber_wallet', None, None, None)
    faber_wallet = await wallet.open_wallet('faber_wallet', None, None)
    ```

8. **Faber** creates a new DID record in its wallet by calling ``did.create_and_store_my_did`` that it will use only for secure interactions with the **Steward**.
    ```python
    # Faber Agent
    (faber_steward_did, faber_steward_key) = await did.create_and_store_my_did(faber_wallet, "{}")
    ```

9. **Faber** creates the connection response which contains the created `DID`, `Verkey` and `Nonce` from the received connection request.
    ```python
    # Faber Agent
    connection_response = json.dumps({
        'did': faber_steward_did,
        'verkey': faber_steward_key,
        'nonce': connection_request['nonce']
    })
    ```

10. **Faber** asks the ledger for the Verification key of the **Steward's** DID by calling ``did.key_for_did``.
    ```python
    # Faber Agent
    steward_faber_verkey = await did.key_for_did(pool_handle, faber_wallet, connection_request['did'])
    ```

11. **Faber** anonymously encrypts the connection response by calling ``crypto.anon_crypt`` with the **Steward** verkey.
   The Anonymous-encryption schema is designed for the sending of messages to a Recipient which has been given its public key. Only the Recipient can decrypt these messages, using its private key. While the Recipient can verify the integrity of the message, it cannot verify the identity of the Sender.
    ```python
    # Faber Agent
    anoncrypted_connection_response = await crypto.anon_crypt(steward_faber_verkey, connection_response.encode('utf-8'))
    ```

12. **Faber** sends the anonymously encrypted connection response to the **Steward**.

13. **Steward** anonymously decrypts the connection response by calling ``crypto.anon_decrypt``.
    ```python
    # Steward Agent
    decrypted_connection_response = \
        (await crypto.anon_decrypt(steward_wallet, steward_faber_key, anoncrypted_connection_response)).decode("utf-8")
    ```

14. **Steward** authenticates **Faber** by the comparison of Nonce.
    ```python
    # Steward Agent
    assert connection_request['nonce'] == decrypted_connection_response['nonce']
    ```

15. **Steward** sends the `NYM` transaction for **Faber's** DID to the Ledger.
Please note that despite the fact that the Steward is the sender of this transaction the owner of the DID will be Faber as it uses the verkey as provided by Faber.
    ```python        
    # Steward Agent
    nym_request = await ledger.build_nym_request(steward_did, decrypted_connection_response['did'], decrypted_connection_response['verkey'], None, role)
    await ledger.sign_and_submit_request(pool_handle, steward_wallet, steward_did, nym_request)
    ```

At this point **Faber** is connected to the **Steward** and can interact in a secure peer-to-peer way. **Faber** can trust the response is from **Steward** because:

* it connects to the current endpoint
* no replay - attack is possible, due to her random challenge
* it knows the verification key used to verify **Steward** digital signature is the correct one because it just confirmed it on the ledger

**Note:** All parties must not use the same DID's to establish other relationships.
By having independent pairwise relationships, you're reducing the ability for others to correlate your activities across multiple interactions.

[previous](03-get-steward-ownership.ipynb) | [next](05-getting-verinym.ipynb)